### The aim of this project is to determine which of 3 West African cities a coffee loving family should stay for a 2 day stopover on their way to South Africa from the United States. The family will like a location where the coffee shops are in close proximity to each other so they can visit as many as possible easily in the limited time they have.

Import Libraries needed

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                       

Obtain Foursquare Credentials

In [2]:
CLIENT_ID = 'UD11RI1VIOZ4EEH0P3KHW3UVGWGNOZEIPH55FD4OL43AFPJ0' # your Foursquare ID
CLIENT_SECRET = '1KMG1T1JJ1AZVO0TF2UR1GPC0MD0HACNYBZX2I3QOWFERHIG' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UD11RI1VIOZ4EEH0P3KHW3UVGWGNOZEIPH55FD4OL43AFPJ0
CLIENT_SECRET:1KMG1T1JJ1AZVO0TF2UR1GPC0MD0HACNYBZX2I3QOWFERHIG


Send the GET request and examine results

In [3]:
LIMIT = 500 # Maximum is 100
cities = ["Lagos", "Abuja", "Accra"]
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d1e0931735") # COFFEE SHOP CATEGORY ID
    results[city] = requests.get(url).json()
results

{'Lagos': {'meta': {'code': 200, 'requestId': '5f074b12c7664158ebec518e'},
  'response': {'suggestedFilters': {'header': 'Tap to show:',
    'filters': [{'name': 'Open now', 'key': 'openNow'}]},
   'geocode': {'what': '',
    'where': 'lagos',
    'center': {'lat': 6.45407, 'lng': 3.39467},
    'displayString': 'Lagos, Nigeria',
    'cc': 'NG',
    'geometry': {'bounds': {'ne': {'lat': 6.6955349502909645,
       'lng': 3.465414995089732},
      'sw': {'lat': 6.394192, 'lng': 3.2203770082172642}}},
    'slug': 'lagos',
    'longId': '72057594040260395'},
   'headerLocation': 'Lagos',
   'headerFullLocation': 'Lagos',
   'headerLocationGranularity': 'city',
   'query': 'coffee shop',
   'totalResults': 27,
   'suggestedBounds': {'ne': {'lat': 6.623755361872141,
     'lng': 3.4767584692164917},
    'sw': {'lat': 6.414956125625136, 'lng': 3.312966395958675}},
   'groups': [{'type': 'Recommended Places',
     'name': 'recommended',
     'items': [{'reasons': {'count': 0,
        'items': [{

In [4]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']
    


Visualize Respective Maps

In [5]:


maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of Coffee shops in {city} = ", results[city]['response']['totalResults'])
    



Total number of Coffee shops in Lagos =  27
Total number of Coffee shops in Abuja =  10
Total number of Coffee shops in Accra =  21


In [6]:
maps[cities[0]] #map of Lagos showing coffee shop locations

In [7]:
maps[cities[1]] #map of Abuja showing coffee shop locations

In [8]:
maps[cities[2]] #map of Accra showing cofffee shop locations

Now find out how densely packed together the coffee shops are in the 3 cities. This we do by calculating the mean loacation in each of the cities and the average mean distance of the coffee shops to that mean. 

In [9]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

Lagos
Mean Distance from Mean coordinates
0.06385700846991325
Abuja
Mean Distance from Mean coordinates
0.01616734229575901
Accra
Mean Distance from Mean coordinates
0.04085511761575744


In [10]:
maps[cities[0]] #Lagos

In [11]:
maps[cities[1]] #Abuja

In [12]:
maps[cities[2]] #Accra

From the analysis of the 3 cities above, it is recommended that the family stay in Abuja for their vacation as Abuja has the least mean distance from mean coordinates which in effect means the coffee shops are bunched closer together therefore making commute time between coffee shops much lesser than the other options.